<a href="https://colab.research.google.com/github/taeheee-kim/deep_learning_practice/blob/master/%EA%B3%B5%EB%AA%A8%EC%A0%84_b6_batchsize6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Oct  5 16:55:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"

albumentations==1.1.0 is successfully installed


## Run the example

### Import the required libraries

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from collections import defaultdict
import copy
import random
import os
import shutil
from urllib.request import urlretrieve

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models

cudnn.benchmark = True

In [5]:
import gc
import torch
import torch.utils.data as Data
import torch.nn as nn
from torchvision import transforms, models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")


# 데이터 경로
data_path = '/content/drive/My Drive/Tomato_version2/'

train = pd.read_csv(data_path + 'train.csv')
#test = pd.read_csv(data_path + 'test.csv')
#submission = pd.read_csv(data_path + 'sample_submission.csv')

import torch # 파이토치
import random
import os


In [6]:

# 시드 값 고정
seed = 10
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
train, valid = train_test_split(train,
                                test_size=0.1,
                                stratify=train[['Tomato_D01','Tomato_D04','Tomato_D05','Tomato_D07','Tomato_D08','Tomato_D09','Tomato_H','Tomato_P03','Tomato_P05','Tomato_R01']],
                                random_state=10)

import cv2
from torch.utils.data import Dataset  # 데이터 생성을 위한 클래스
import numpy as np
import glob


class ImageDataset(Dataset):
    # 초기화 메서드(생성자)
    def __init__(self, df, img_dir='./', transform=None, is_test=False):
        super().__init__()  # 상속받은 Dataset의 __init__() 메서드 호출
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test

    # 데이터 세트 크기 반환 메서드
    def __len__(self):
        return len(self.df)

    # 인덱스(idx)에 해당하는 데이터 반환 메서드
    def __getitem__(self, idx):
        
        img_id = self.df.iloc[idx, 0] # 이미지 ID
        img_path = self.img_dir + str(img_id) + '.png' # 이미지 파일 경로
        image = cv2.imread(img_path) # 이미지 파일 읽기
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 이미지 색상 보정
        
        # 이미지 변환
        if self.transform is not None:
            image = self.transform(image=image)['image']


        # 테스트 데이터면 이미지 데이터만 반환, 그렇지 않으면 타깃 값(label)도 함께 반환
        if self.is_test:
            return image
        else:
            # 타깃 값 4개 중 가장 큰 값의 인덱스
            label = np.argmax(self.df.iloc[idx, 1:11])
            return image, label

In [7]:
# 이미지 변환을 위한 모듈
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 훈련 데이터용 변환기
transform_train = A.Compose([
    A.Resize(400, 600), # 이미지 크기 조절
    # 밝기 및 대비 조절 
    A.RandomBrightnessContrast(brightness_limit=0.1, 
                               contrast_limit=0.1, p=0.5),
    A.VerticalFlip(p=0.5), # 상하 대칭 변환
    A.HorizontalFlip(p=0.5), # 좌우 대칭 변환
    # 이동, 스케일링, 회전 변환 
    A.ShiftScaleRotate(
        shift_limit=0.1,
        scale_limit=0.2,
        rotate_limit=25, p=0.7),
    # 양각화, 날카로움, 블러 효과 
    A.OneOf([A.Emboss(p=1),
             A.Sharpen(p=1),
             A.Blur(p=1)], p=0.5),
    A.PiecewiseAffine(p=0.5), # 어파인 변환 
    A.Normalize(), # 정규화 변환 
    ToTensorV2() # 텐서로 변환
])

# 검증 및 테스트 데이터용 변환기
transform_test = A.Compose([
    A.Resize(400, 600),
    A.Normalize(),
    ToTensorV2()
])

In [8]:
img_dir = '/content/drive/My Drive/Tomato_version2/images/'

dataset_train = ImageDataset(train, img_dir=img_dir, transform=transform_train)
dataset_valid = ImageDataset(valid, img_dir=img_dir, transform=transform_test)

In [9]:
def seed_worker(worker_id): #시드를 고정해주는 역할
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
g = torch.Generator()
g.manual_seed(0)

In [10]:
from torch.utils.data import DataLoader # 데이터 로더 생성을 위한 클래스

batch_size = 6

loader_train = DataLoader(dataset_train, batch_size=batch_size, 
                          shuffle=True, worker_init_fn=seed_worker,
                          generator=g)
loader_valid = DataLoader(dataset_valid, batch_size=batch_size, 
                          shuffle=False, worker_init_fn=seed_worker,
                          generator=g)

In [ ]:
!pip install efficientnet-pytorch==0.7.1

In [11]:
!pip install transformers

In [12]:
from efficientnet_pytorch import EfficientNet # EfficientNet 모델

# 사전 훈련된 efficientnet-b7 모델 불러오기

model = EfficientNet.from_pretrained('efficientnet-b6', num_classes=10) 

model = model.to(device) # 장비 할당

Loaded pretrained weights for efficientnet-b6


In [13]:
import torch.nn as nn # 신경망 모듈

# 손실함수
criterion = nn.CrossEntropyLoss()

# 옵티마이저
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00007, weight_decay=0.0001)

from transformers import get_cosine_schedule_with_warmup

epochs = 40 # 총 에폭

# 스케줄러
scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=len(loader_train)*5, 
                                            num_training_steps=len(loader_train)*epochs)


In [14]:
train_loss = []
valid_loss = []
train_acc = []
val_acc = []

In [15]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수
from tqdm.notebook import tqdm # 진행률 표시 막대

# 총 에폭만큼 반복
for epoch in range(epochs):
    model.train() # 모델을 훈련 상태로 설정
    epoch_train_loss = 0 # 에폭별 손실값 초기화 (훈련 데이터용)

    # '반복 횟수'만큼 반복 
    for images, labels in tqdm(loader_train):
    
        # 이미지, 레이블(타깃 값) 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device)
        
        # 옵티마이저 내 기울기 초기화
        optimizer.zero_grad()
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 손실함수를 활용해 outputs와 labels의 손실값 계산
        loss = criterion(outputs, labels)
        # 현재 배치에서의 손실 추가 (훈련 데이터용)
        epoch_train_loss += loss.item() 
        # 역전파 수행
        loss.backward() 
        # 가중치 갱신
        optimizer.step() 
        # 스케줄러 학습률 갱신
        scheduler.step() 
        
    if (epoch+1)%5==0: #5 10,15 20, 30, ... 
          path = '/content/drive/My Drive/Tomato_version2/태희/epoch' + str(epoch) + '.pt'
          torch.save(model.state_dict(), path)

    # 훈련 데이터 손실값 출력
    print(f'에폭 [{epoch+1}/{epochs}] - 훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
    
    model.eval() # 모델을 평가 상태로 설정 
    epoch_valid_loss = 0 # 에폭별 손실값 초기화 (검증 데이터용) 
    preds_list = [] # 예측 확률값 저장용 리스트 초기화 
    true_onehot_list = [] # 실제 타깃 값 저장용 리스트 초기화 
    
    with torch.no_grad(): # 기울기 계산 비활성
        for images, labels in loader_valid:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            epoch_valid_loss += loss.item()
            
            preds = torch.softmax(outputs.cpu(), dim=1).numpy() # 예측 확률값
            true_onehot = torch.eye(10)[labels].cpu().numpy() # 실제값 (원-핫 인코딩 형식)
            # 예측 확률값과 실제값 저장
            preds_list.extend(preds)
            true_onehot_list.extend(true_onehot)
            
        # 검증 데이터 손실값 및 ROC AUC 점수 출력 
        print(f'에폭 [{epoch+1}/{epochs}] - 검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f} / 검증 데이터 ROC AUC : {roc_auc_score(true_onehot_list, preds_list):.4f}')  


  0%|          | 0/536 [00:00<?, ?it/s]

에폭 [1/40] - 훈련 데이터 손실값 : 2.1907
에폭 [1/40] - 검증 데이터 손실값 : 1.8510 / 검증 데이터 ROC AUC : 0.8405


  0%|          | 0/536 [00:00<?, ?it/s]

에폭 [2/40] - 훈련 데이터 손실값 : 1.3092
에폭 [2/40] - 검증 데이터 손실값 : 0.5447 / 검증 데이터 ROC AUC : 0.9663


  0%|          | 0/536 [00:00<?, ?it/s]

...

에폭 [38/40] - 훈련 데이터 손실값 : 0.0070
에폭 [38/40] - 검증 데이터 손실값 : 0.0472 / 검증 데이터 ROC AUC : 0.9997


  0%|          | 0/536 [00:00<?, ?it/s]

에폭 [39/40] - 훈련 데이터 손실값 : 0.0028
에폭 [39/40] - 검증 데이터 손실값 : 0.0464 / 검증 데이터 ROC AUC : 0.9997


  0%|          | 0/536 [00:00<?, ?it/s]

에폭 [40/40] - 훈련 데이터 손실값 : 0.0041
에폭 [40/40] - 검증 데이터 손실값 : 0.0457 / 검증 데이터 ROC AUC : 0.9997


In [16]:
#PATH = '/content/drive/My Drive/Tomato_version2/'

#모델 저장하기
#torch.save(model, PATH + 'model.pt')

In [17]:
#테스트 사진으로 평가 후 csv 채워넣기

In [18]:
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [19]:
img_dir = '/content/drive/My Drive/Tomato_version2/test_images/'

dataset_TTA = ImageDataset(test, img_dir=img_dir, 
                           transform=transform_train, is_test=True)
loader_TTA = DataLoader(dataset_TTA, batch_size=batch_size, 
                        shuffle=False, worker_init_fn=seed_worker,
                        generator=g)

In [20]:
num_TTA = 5 # TTA 횟수

preds_tta = np.zeros((len(test), 10)) # 예측 값 저장용 배열 초기화 (TTA용)

# TTA를 적용해 예측 (5에폭)
for i in range(num_TTA):
    with torch.no_grad():
        for i, images in enumerate(loader_TTA):
            images = images.to(device)
            outputs = model(images)
            # 타깃 예측 확률
            preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy()
            preds_tta[i*batch_size:(i+1)*batch_size] += preds_part

In [21]:
preds_tta /= num_TTA 

In [22]:
submission_tta = submission.copy() 

submission_tta[['Tomato_D01','Tomato_D04','Tomato_D05','Tomato_D07','Tomato_D08','Tomato_D09','Tomato_H','Tomato_P03','Tomato_P05','Tomato_R01']] = preds_tta

In [23]:
submission_tta

,image_id,Tomato_D01,Tomato_D04,Tomato_D05,Tomato_D07,Tomato_D08,Tomato_D09,Tomato_H,Tomato_P03,Tomato_P05,Tomato_R01
0,0.png,9.994364e-01,4.513194e-04,5.614660e-05,1.114929e-05,4.435134e-06,2.132385e-05,3.005234e-06,4.148979e-07,3.930895e-06,1.180520e-05
1,1.png,5.337654e-07,6.202123e-05,9.999124e-01,7.858530e-06,1.565983e-06,2.606365e-06,8.391840e-06,6.166177e-07,3.322532e-06,6.948754e-07
2,2.png,1.452513e-04,1.120518e-05,4.146183e-06,4.102146e-06,6.846769e-07,8.830254e-06,9.997961e-01,8.748457e-07,1.173994e-05,1.702230e-05
3,3.png,2.956523e-08,9.999973e-01,4.297027e-07,2.382678e-08,2.345541e-07,2.045390e-07,1.739388e-06,5.970971e-08,2.951413e-08,7.779479e-08
4,4.png,3.157379e-08,4.089535e-06,9.999811e-01,2.654324e-08,7.248736e-07,1.735511e-08,2.743602e-06,2.610273e-09,1.120984e-05,9.404715e-08
...,...,...,...,...,...,...,...,...,...,...,...
3458,3458.png,9.999872e-01,4.472571e-06,2.778890e-06,2.932128e-07,2.540190e-07,3.020286e-07,5.403004e-07,2.066510e-07,3.647566e-06,3.278080e-07
3459,3459.png,5.292577e-06,1.766405e-05,1.051527e-07,2.209217e-07,4.451820e-06,1.239253e-06,9.999673e-01,2.961704e-06,5.285494e-07,3.277644e-07
3460,3460.png,1.896144e-06,1.570778e-05,3.397902e-07,1.004077e-06,3.278103e-07,2.459007e-06,8.910963e-05,9.998869e-01,2.158245e-06,1.647695e-07
3461,3461.png,4.058666e-06,1.534304e-05,2.537790e-08,7.551618e-07,3.838852e-07,1.786283e-07,9.999774e-01,7.597342e-07,7.383087e-07,3.366212e-07


In [25]:
id = submission_tta["image_id"]

In [26]:
pred_idx = np.argmax(preds_tta, axis=1)  #확률 중 가장 높은 값의 인덱스를 0 ~ 9까지 반환

In [27]:
pred_idx

array([0, 2, 6, ..., 7, 6, 6])

In [28]:
df = pd.DataFrame({'file_name':id, 'answer':pred_idx})

In [29]:
df 

,file_name,answer
0,0.png,0
1,1.png,2
2,2.png,6
3,3.png,1
4,4.png,2
...,...,...
3458,3458.png,0
3459,3459.png,6
3460,3460.png,7
3461,3461.png,6


In [30]:
df.to_csv('/content/drive/My Drive/Tomato_version2/태희/submission_tta.csv', index=False)